In [12]:
import pandas as pd
import numpy as np
import os
import json
import altair as alt
import numpy as np

In [13]:
JSON_FILE = os.path.join("H:\\", "AOC-REM2", "results", "mammalian_REM2", "mammalian_REM2_codons.SA.fasta.BUSTEDS.json")
pvalueThreshold = 0.05

In [14]:
def getBUSTEDS_ER(json_file):
    with open(json_file, "r") as in_d:
        json_data = json.load(in_d)
    return json_data["Evidence Ratios"]
#end method

def getBUSTEDS_TestResults(json_file):
    with open(json_file, "r") as in_d:
        json_data = json.load(in_d)
    return json_data["test results"]
#end method

def getBUSTEDS_Fits(json_file):
    with open(json_file, "r") as in_d:
        json_data = json.load(in_d)
    return json_data["fits"]
#end method

## ER Plots


In [15]:
data      = getBUSTEDS_ER(JSON_FILE)
num_sites =  len(data["constrained"][0])

res = [x+1 for x in range(num_sites)]
data_tuples = list(zip(res,data["constrained"][0]))
df = pd.DataFrame(data_tuples, columns=['Site','ER'])
df["2ln(ER)"] = 2 * np.log(df["ER"])
df

,Site,ER,2ln(ER)
0,1,4.393329,2.960175e+00
1,2,5.391779,3.369751e+00
2,3,1.000000,0.000000e+00
3,4,1.000000,2.220446e-15
4,5,1.000000,-2.664535e-15
...,...,...,...
651,652,0.571549,-1.118810e+00
652,653,0.659019,-8.340047e-01
653,654,0.568913,-1.128054e+00
654,655,14.934769,5.407384e+00


In [16]:
#source = df[df["omega"] < 10]
source = df

line = alt.Chart(source).mark_line().encode(
    x='Site',
    y='2ln(ER)', 
).properties(
    width=800,
    height=600)

line

alt.Chart(...)

## Fits plots

In [17]:
# Unconstrained model

data   = getBUSTEDS_Fits(JSON_FILE)
df_Unc = pd.DataFrame.from_dict(data["Unconstrained model"]["Rate Distributions"]["Test"], orient='index')

source = df_Unc

line = alt.Chart(source).mark_bar().encode(
    x='sum(proportion)',
    y='proportion',
    color=alt.Color('omega', scale=alt.Scale(scheme='reds', reverse=False))
).properties(
    width=500,
    height=400)

line

alt.Chart(...)

In [18]:
data   = getBUSTEDS_Fits(JSON_FILE)
df_Unc = pd.DataFrame.from_dict(data["Unconstrained model"]["Rate Distributions"]["Test"], orient='index')
df_Unc["proportion"] = round(df_Unc["proportion"] * 100, 2)
if df_Unc["omega"][2] > 10000: 
    df_Unc["omega"][2] = 10000
df_Unc

,omega,proportion
0,0.041782,88.68
1,1.000000,10.88
2,38.453045,0.44


In [19]:
print(df_Unc["omega"][2])
if df_Unc["omega"][2] > 10000: 
    df_Unc["omega"][2] = 10000

38.45304497265218


In [20]:
# Unconstrained model

#data   = getBUSTEDS_Fits(JSON_FILE)
#df_Unc = pd.DataFrame.from_dict(data["Unconstrained model"]["Rate Distributions"]["Test"], orient='index')

source = df_Unc

bar = alt.Chart(source).mark_bar().encode(
    x='omega',
    y='proportion',
    color=alt.Color('omega', scale=alt.Scale(scheme='reds', reverse=False))
).properties(
    width=800,
    height=400)


text = bar.mark_text(
    align='left',
    baseline='middle',
    color='black',
    dx=3  # Nudges text to right so it doesn't appear on top of the bar
).encode(
    text='proportion:Q'
)

bar + text

#bar.save('Figure1_BUSTEDS.png')


alt.LayerChart(...)

In [21]:
# Figure Legend

result = getBUSTEDS_TestResults(JSON_FILE)
print(result)

result_response = ""

if result["p-value"] <= pvalueThreshold:
    result_response = "found evidence (LRT, p-value ≤ 0.05)"
else:
    result_response = "did NOT find evidence (LRT, p-value > 0.05)"


#msg = "BUSTEDS found evidence " + result_response + " of gene-wide episodic diversifying selection in the selected test branches of your phylogeny."

msg = "BUSTEDS analysis " + result_response + " of gene-wide episodic diversifying selection in the selected test branches of our phylogeny. "


print("\n" + msg)

{'LRT': 166.1420432398299, 'p-value': 0}

BUSTEDS analysis found evidence (LRT, p-value ≤ 0.05) of gene-wide episodic diversifying selection in the selected test branches of our phylogeny. 
